<a href="https://colab.research.google.com/github/amitkp57/colab/blob/main/MLiB_L4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Libraries

In [ ]:
!pip install tokenizer path tokenizers

     |████████████████████████████████| 3.3 MB 14.0 MB/s 


## Data





In [ ]:
# import from google drive
# run this code piece, enter the autorization code
# For mount instructions: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd

from google.colab import drive
drive.mount("/content/drive")

# or import from environment
PROJECT_DIR = "/content/drive/My Drive/MLBiology/MLiB-Lab4"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '$PROJECT_DIR'
!unzip '$PROJECT_DIR/MLiB-Lab4.zip' -d /content/GPT-J

/content/drive/My Drive/MLBiology/MLiB-Lab4
Archive:  /content/drive/My Drive/MLBiology/MLiB-Lab4/MLiB-Lab4.zip
mapname:  conversion of  failed
   creating: /content/GPT-J/GPTJ-CP/
   creating: /content/GPT-J/GPT-J-Run-Output/
 extracting: /content/GPT-J/GPTJ-CP/query.py  
 extracting: /content/GPT-J/install_gpt-j.sh  
 extracting: /content/GPT-J/MLiB-Lab4-GPT-Models.pdf  
 extracting: /content/GPT-J/MLiB-Lab4-GPT-Models.docx  
 extracting: /content/GPT-J/GPTJ-CP/run-query-loop.py  
 extracting: /content/GPT-J/GPTJ-CP/convert-n-run-query-loop.py  
 extracting: /content/GPT-J/GPT-J-Run-Output/drug_prompt_output.txt  
 extracting: /content/GPT-J/GPT-J-Run-Output/gene_product_output.txt  
 extracting: /content/GPT-J/GPT-J-Run-Output/drug_targets_output.txt  
 extracting: /content/GPT-J/GPT-J-Run-Output/names_patents_output.txt  
 extracting: /content/GPT-J/GPT-J-Run-Output/names_aliases_output.txt  
 extracting: /content/GPT-J/GPT-J-Run-Output/drug_inhibits_output.txt  
 extracting: /cont

In [ ]:
%cd /content/GPT-J
!bash install_gpt-j.sh

/content/GPT-J
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.7 MB 8.5 kB/s 
     |████████████████████████████████| 17.3 MB 2.9 MB/s 
     |████████████████████████████████| 1.9 MB 5.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0

Parse the output prompt and response files. Sample 10 data point from each of the files and store them.

In [ ]:
def parse_prompt_reponse(f):
  prompt_response = []
  while True:
      next_line = f.readline()
      if next_line == '':
        break
      elif next_line.startswith('Enter prompt or quit:'):
        next_line = next_line[len('Enter prompt or quit:'):].strip(' ')
      if next_line.startswith('Prompt:'):
        prompt = next_line[len('Prompt:'):].strip()
        next_line = f.readline()
        response = []
        while not next_line.startswith('Enter prompt or quit:'):
          if next_line.startswith('Response:'):
            response.append(next_line[len('Response:'):].strip())
          else:
            response.append(next_line)
          next_line = f.readline()
        prompt_response.append((prompt, ' '.join(response).strip()))
  return prompt_response

In [ ]:
import numpy as np
import os

# sample 10 data point from each of the files

for f_name in os.listdir('/content/GPT-J/GPT-J-Run-Output'):
  file_path = f'/content/GPT-J/GPT-J-Run-Output/{f_name}'
  print(file_path)
  f = open(file_path, 'r')
  prompt_response = parse_prompt_reponse(f)
  f.close()
  prompt_response = np.array(prompt_response)
  samples = prompt_response[np.random.choice(range(len(prompt_response)), 10)]
  # sample 10 items
  with open(f'{PROJECT_DIR}/results/{f_name}', 'w') as f:
    for sample in samples:
      prompt, response = sample
      f.write('prompt: {}\n'.format(prompt))
      f.write('response: {}\n'.format(response))

/content/GPT-J/GPT-J-Run-Output/drug_inhibits_output.txt
/content/GPT-J/GPT-J-Run-Output/drug_targets_output.txt
/content/GPT-J/GPT-J-Run-Output/drug_prompt_output.txt
/content/GPT-J/GPT-J-Run-Output/names_aliases_output.txt
/content/GPT-J/GPT-J-Run-Output/drug_mechanism_of_action_output.txt
/content/GPT-J/GPT-J-Run-Output/names_patents_output.txt
/content/GPT-J/GPT-J-Run-Output/gene_product_output.txt


Sample 100 drugs and for each drug combine and save drug inhibition, mechanism and targers.

In [ ]:
import numpy as np
# take 100 points each from 'drug_inhibits_output.txt', 'drug_mechanism_of_action_output.txt', 'drug_targets_output.txt'

# select 100 drugs
sample_indices = np.random.choice(range(2835), 100)
file_path = f'/content/GPT-J/GPT-J-Run-Output/drug_inhibits_output.txt'
f = open(file_path, 'r')
prompt_response = parse_prompt_reponse(f)
f.close()
prompt_response = {prompt: reponse for (prompt, reponse) in prompt_response}

drugs = prompt_response.keys()
drugs = map(lambda x: x[:-len('is a drug that inhibits')], drugs)
drugs = np.array(list(drugs))[sample_indices]
print(drugs)

['norethindrone' 'pimozide' 'Hyoscyamine' 'S4689' 'INCB28060'
 'Oxtriphylline' 'L-Lactic' 'S5234' 'S5086' 'MK-2206' 'S5505'
 'Molidustat(BAY' 'S4661' 'Fingolimod' 'S4668' 'Reboxetine' 'Ostarine'
 'S5473' 'S4727' 'S3950' 'S5483' 'Prucalopride' 'Avanafil' 'Propranolol'
 'S3650' 'S3969' 'LY3039478' 'Octanoic' 'S3761' 'Lumiracoxib' 'GSK1265744'
 'Famciclovir' 'SAR125844' 'PENETREX' 'Ceftizoxime' 'S4870' 'Undecanoic'
 'S3772' 'Ropivacaine' 'S3951' 'Tanshinone' 'Fumalic' 'LXS-196'
 'Flumequine' 'TAK-117' 'Genistein' 'S9102' 'S5528' 'PD-0332991' 'S4059'
 'MK-2206' 'RAPAMYCIN' 'S5592' 'Oxcarbazepine' 'LCI699' 'Diphenidol'
 'Oltipraz' 'Oclacitinib' 'S5157' 'ALDOCLOR-250' 'S5629' 'L-NAME'
 'Pentamidine' 'CETACORT' 'Pancuronium' 'BENOQUIN' 'S5627' 'Fulvestrant'
 'Puerarin' 'S5581' 'S5302' 'GS-9620' 'S5398' 'Trospium' 'S4657'
 'Demecarium' 'Tranexamic' 'estradiol' 'Brinzolamide' 'PLX-4032'
 'nifuroxazide' 'AZOLID' 'Fluralaner' 'Oclacitinib' 'S5626' 'Estramustine'
 'Rebamipide' 'Daidzein' 'S3745' '

In [ ]:
from collections import defaultdict
drug_fn_map = {
    'drug_inhibits_output.txt': lambda x :  x.strip().split()[0][:-2], #S4819is a drug that inhibits
    'drug_mechanism_of_action_output.txt': lambda x :  x.strip().split()[-2], # The mechanism of action of S4819 is
    'drug_targets_output.txt': lambda x :  x.strip().split()[0] # S4637 is a drug that targets
}


drug_data = defaultdict(list)
for f_name in ['drug_inhibits_output.txt', 'drug_mechanism_of_action_output.txt', 'drug_targets_output.txt']:
  file_path = f'/content/GPT-J/GPT-J-Run-Output/{f_name}'
  f = open(file_path, 'r')
  prompt_response = parse_prompt_reponse(f)
  f.close()
prompt_response = {prompt: reponse for (prompt, reponse) in prompt_response}

with open(f'{PROJECT_DIR}/results/drug_100.txt', 'w') as f:    
  for drug in drug_data.keys():
    inhibit, mechanism, target = drug_data[drug]
    f.write(f'{drug}: \n')
    f.write(f'inhibits: {inhibit}\n')
    f.write(f'mechanism: {mechanism}\n')
    f.write(f'target: {target}\n')

Create a bag of words model from the text in the 7 files. We will use a BPE tokenizer with 30,000 tokens as vocabulary. 

In [ ]:
from path import Path
from sklearn import preprocessing
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

In [ ]:
def save_tokenizer(corpus_path, tokenizer_path):
    tokenizer = Tokenizer(BPE())
    tokenizer.pre_tokenizer = Whitespace()
    trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=30000)
    tokenizer.train(files=list(Path(corpus_path).walkfiles('*.txt')), trainer=trainer)
    tokenizer.save(tokenizer_path)
    return


def load_tokenizer(tokenizer_path):
    return Tokenizer.from_file(tokenizer_path)

save_tokenizer('/content/GPT-J/GPT-J-Run-Output', '{}/tokenizer.txt'.format(PROJECT_DIR))
tokenizer = load_tokenizer('{}/tokenizer.txt'.format(PROJECT_DIR))

In [ ]:
def get_bow_vector(tokenizer, input):
  ids = tokenizer.encode(input).ids 
  output = np.zeros((1, 30000))
  for id in ids:
    output[0][id] = 1
  return output

In [ ]:
tokenizer = load_tokenizer('{}/tokenizer.txt'.format(PROJECT_DIR))

# gene products
file_path = f'/content/GPT-J/GPT-J-Run-Output/gene_product_output.txt'
f = open(file_path, 'r')
gene_product = parse_prompt_reponse(f)
f.close()

gene_product = list(map(lambda x: (x[0].split()[3], x[1]), gene_product))
gene_product_tokens = list(map(lambda x: (x[0], get_bow_vector(tokenizer, x[1])), gene_product))

# drug targets
file_path = f'/content/GPT-J/GPT-J-Run-Output/drug_targets_output.txt'
f = open(file_path, 'r')
drug_target = parse_prompt_reponse(f)
f.close()

drug_target = list(map(lambda x: (x[0].split()[0], x[1]), drug_target))
drug_target_tokens = list(map(lambda x: (x[0], get_bow_vector(tokenizer, x[1])), drug_target))

In [ ]:
# patents
file_path = f'/content/GPT-J/GPT-J-Run-Output/names_patents_output.txt'
f = open(file_path, 'r')
drug_patents = parse_prompt_reponse(f)
f.close()

drug_patents = list(map(lambda x: (x[0].split()[0], x[1]), drug_patents))
drug_patents_tokens = list(map(lambda x: (x[0], get_bow_vector(tokenizer, x[1])), drug_patents))

In [ ]:
from scipy import spatial

gene_drug = []
for i in range(min(100, len(gene_product))):
  max_score, max_idx = 0, 0
  for j in range(len(drug_target)):
    similarity = 1 - spatial.distance.cosine(gene_product_tokens[i][1], drug_target_tokens[j][1])
    if similarity > max_score:
      max_score = similarity
      max_idx = j
  gene_drug.append((i,max_idx))

In [ ]:
from scipy import spatial

drug_patent = []
for i in range(min(100, len(drug_target))):
  max_score, max_idx = 0, 0
  for j in range(len(drug_patents)):
    similarity = 1 - spatial.distance.cosine(drug_target_tokens[i][1], drug_patents_tokens[j][1])
    if similarity > max_score:
      max_score = similarity
      max_idx = j
  drug_patent.append((i,max_idx))

In [ ]:
import re

def get_gene_product(response):
  pat = re.compile(r'(The gene product of[^\.!?]*[\.!?])', re.M)
  return pat.findall(response)[0].split(maxsplit=7)[-1].strip(' .')

# def get_drug_product(response):
#   print(response)
#   idx = response.find('is a drug that targets')
#   return pat.findall(response)[:idx]

gene_drug_prod = []
for i in range(len(gene_drug)):
  try:
    drug = drug_target[gene_drug[i][1]][0]
    gene = gene_product[gene_drug[i][0]][0]
    prod = get_gene_product(gene_product[gene_drug[i][0]][1])
  except Exception as e:
    print(e)
    continue
  gene_drug_prod.append((gene, prod, drug))

In [ ]:
import pandas as pd

pd.DataFrame(gene_drug_prod, columns=['gene', 'product', 'drug'])

,gene,product,drug
0,symbol,[,S5248
1,A1BG,glycoprotein,RVX-208(RVX-000222)
2,A1BG-AS1,antisense RNA that may regulate A1BG (alpha-1B...,CC-223
3,A1CF,"novel cytidine deaminase, which is a member of...",Sulfamerazine
4,A2M,"which is a multifunctional,\n bioactive, plasm...",NAQUIVAL
...,...,...,...
91,ABCD4,member of the family of ATP-binding cassette (...,PA-824
92,ABCE1,membrane transporter protein of the ATP-bindin...,PA-824
93,ABCF1,"human ortholog of the yeast RAD1 gene, which e...",S5496
94,ABCF1-DT,protein,RVX-208(RVX-000222)


In [ ]:
import re

def get_gene_product(response):
  pat = re.compile(r'(The gene product of[^\.!?]*[\.!?])', re.M)
  return pat.findall(response)[0].split(maxsplit=7)[-1].strip(' .')

# def get_drug_product(response):
#   print(response)
#   idx = response.find('is a drug that targets')
#   return pat.findall(response)[:idx]

gene_drug_prod = []
for i in range(len(gene_drug)):
  try:
    drug = drug_target[gene_drug[i][1]][0]
    gene = gene_product[gene_drug[i][0]][0]
    prod = get_gene_product(gene_product[gene_drug[i][0]][1])
    patent = drug_patents[gene_drug[i][1]]
  except Exception as e:
    print(e)
    continue
  gene_drug_prod.append((gene, prod, drug, patent))

In [ ]:
import pandas as pd

pd.DataFrame(gene_drug_prod, columns=['gene', 'product', 'drug', 'patent'])

,gene,product,drug,patent
0,symbol,[,S5248,"(Cyproterone, Cyproterone is a drug mentioned ..."
1,A1BG,glycoprotein,RVX-208(RVX-000222),"(CARBAMAZEPINE, CARBAMAZEPINE is a drug mentio..."
2,A1BG-AS1,antisense RNA that may regulate A1BG (alpha-1B...,CC-223,"(2-Pyridylacetonitrile, 2-Pyridylacetonitrile ..."
3,A1CF,"novel cytidine deaminase, which is a member of...",Sulfamerazine,"(Fluralaner, Fluralaner is a drug mentioned in..."
4,A2M,"which is a multifunctional,\n bioactive, plasm...",NAQUIVAL,"(Ammonium, Ammonium is a drug mentioned in pat..."
...,...,...,...,...
147,ABCD1P4,ATP-binding cassette transporter protein that ...,Macitentan,"(AVC, AVC is a drug mentioned in patent litera..."
148,ABCD4,member of the family of ATP-binding cassette (...,S5630,"(EAI045, EAI045 is a drug mentioned in patent ..."
149,ABCE1,membrane transporter protein of the ATP-bindin...,S3892,"(CL-387785, CL-387785 is a drug mentioned in p..."
150,ABCF1,"human ortholog of the yeast RAD1 gene, which e...",MK-2206,"(Alfacalcidol, Alfacalcidol is a drug mentione..."


# Writeup

I sampled 10 prompt and responses from each of the 7 files each. I tried to verify the truth of responses. I could not verify most of the responses. It looked like the response was completedly made using similar meaning words from different references. 

